In [1]:
from flask import Flask, render_template, request
from impala.dbapi import connect
import mlflow
import pandas as pd
import json
import os

import numpy as np

#testing w / o sql alchemy for the moment
#import sqlalchemy as db
#from sqlalchemy import Table, MetaData, select, func, text

import pickle
import numpy
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
from random import randrange

In [3]:
import joblib

**load one hot encodeing pickly**

In [4]:
ohefit = joblib.load("data/ohefit.save")

**load model**

In [5]:
# Load the model from the file in another script/notebook
with open('models_from_code/random_forest_model.pkl', 'rb') as model_file:
    rf_loaded = pickle.load(model_file)

In [6]:
#/home/cdsw/.experiments/42nu-7zr9-wubg-27kl', creation_time=None, experiment_id='42nu-7zr9-wubg-27kl'

### Load pickle files

In [7]:
with open('data/standardscaler.pkl', 'rb') as file:
    loaded_standardscaler = pickle.load(file)

# with open('data/labelencoder.pkl', 'rb') as file:
#     loaded_labelencoder = pickle.load(file)

with open('data/data_num_cols.pkl', 'rb') as file:
    data_num_cols = pickle.load(file)

with open('data/data_cat_cols.pkl', 'rb') as file:
    data_cat_cols = pickle.load(file)

with open('data/onehotenc.pkl','rb') as file:
    ohefit = pickle.load(file)

### Normalize function

In [8]:
def normalize_data(raw_data):
    print(f"top raw_data = {raw_data}\n")
    #raw_data='{"Age": 27, "Vintage": 26, "Avg_Account_Balance": 707906, "Channel_Code": "X1", "Credit_Product": "No", "Gender": "Female", "Is_Active": "No", "Occupation": "Salaried", "Region_Code": "RG256"}'  
    
    #data = json.loads(raw_data)
    #print(f"json raw_data = {[raw_data]}")

    cc_vector_df = pd.DataFrame([raw_data])    
    print(f"raw pandas dataframe {cc_vector_df}\n")    
        
    data_num_data = cc_vector_df.loc[:, data_num_cols]
    data_cat_data = cc_vector_df.loc[:, data_cat_cols]
    
    #print("Shape of num data:", data_num_data.shape)
    #print("Shape of cat data:", data_cat_data.shape)
    
    print(f"data_num_data: {data_num_data}\n")
    print(f"data_cat_data: {data_cat_data}\n")

    data_num_data['Avg_Account_Balance'] = np.log(data_num_data['Avg_Account_Balance'])
        
    #data_num_data_s = loaded_standardscaler.fit_transform(data_num_data)
    data_num_data_s = loaded_standardscaler.transform(data_num_data)
    data_num_data_df_s = pd.DataFrame(data_num_data_s, columns = data_num_cols)

    #print('numerical test data post standard scaler', data_num_data_s)
   
    data_cat_data_norm = ohefit.transform(data_cat_data)    

    #print('columns of categorical encoded df', data_cat_data_norm.columns)
    #print('type of categorical encoded df', type(data_cat_data_norm))

    #print (' catagorical data post encoding', data_cat_data_norm)
  
    data_new = pd.concat([data_num_data_df_s, data_cat_data_norm], axis = 1)
    
    print(f"transformed data={data_new}\n")
    return data_new

**Test Data**

In [9]:
test_data = pd.read_csv('test_data_file.csv')

In [13]:
row_number = randrange(100)

In [14]:
test_sample = test_data.iloc[row_number].to_dict()

In [15]:
test_sample

{'ID': 'CTUXJGBS',
 'Gender': 'Female',
 'Age': 53,
 'Region_Code': 'RG284',
 'Occupation': 'Self_Employed',
 'Channel_Code': 'X3',
 'Vintage': 109,
 'Credit_Product': 'Not Sure',
 'Avg_Account_Balance': 2322802,
 'Is_Active': 'Yes'}

In [21]:
norm_df  = normalize_data(test_sample)

top raw_data = {'ID': 'X4JOAPHA', 'Gender': 'Male', 'Age': 67, 'Region_Code': 'RG261', 'Occupation': 'Other', 'Channel_Code': 'X1', 'Vintage': 19, 'Credit_Product': 'No', 'Avg_Account_Balance': 1013852, 'Is_Active': 'No'}

raw pandas dataframe          ID Gender  Age Region_Code Occupation Channel_Code  Vintage  \
0  X4JOAPHA   Male   67       RG261      Other           X1       19   

  Credit_Product  Avg_Account_Balance Is_Active  
0             No              1013852        No  

data_num_data:    Age  Vintage  Avg_Account_Balance
0   67       19              1013852

data_cat_data:   Channel_Code Credit_Product Gender Is_Active Occupation Region_Code
0           X1             No   Male        No      Other       RG261

transformed data=       Age   Vintage  Avg_Account_Balance  Channel_Code_X1  Channel_Code_X2  \
0  1.55905 -0.863745             0.153121              1.0              0.0   

   Channel_Code_X3  Channel_Code_X4  Credit_Product_No  \
0              0.0            

In [22]:
norm_df.head()

,Age,Vintage,Avg_Account_Balance,Channel_Code_X1,Channel_Code_X2,Channel_Code_X3,Channel_Code_X4,Credit_Product_No,Credit_Product_Not Sure,Credit_Product_Yes,...,Region_Code_RG275,Region_Code_RG276,Region_Code_RG277,Region_Code_RG278,Region_Code_RG279,Region_Code_RG280,Region_Code_RG281,Region_Code_RG282,Region_Code_RG283,Region_Code_RG284
0,1.55905,-0.863745,0.153121,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### make predictions

In [ ]:
#rf_loaded

In [23]:
rf_pred = rf_loaded.predict(norm_df)

In [25]:
rf_pred

array([0], dtype=int32)

In [24]:
# prediction = loaded_model.predict(norm_df)
# print(f"prediction = {prediction[0]}")

NameError: name 'loaded_model' is not defined